In [8]:
import json
import re
import pandas as pd
import nltk
import math
from math import sqrt
import sklearn
import statsmodels.api as sm
from nltk.stem import WordNetLemmatizer 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
from tqdm import tqdm
from sklearn import linear_model
import collections
from gensim.models import Word2Vec
from gensim.models import word2vec
import multiprocessing
import time
import re
import csv
import numpy as np

In [9]:
#Read review_train.json file
rv_train = []
with open('review_train.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tra = json.loads(line)
                    rv_train.append(rv_tra)
                else:
                    break
        except:
            f.close()
rv_train = pd.DataFrame(rv_train)

In [26]:
# Read the business_train file.
business_train = pd.read_csv("business_train.csv", index_col = 0)

In [79]:
id_gym = []
id_review = rv_train['business_id']
id_business = list(business_train['business_id'])
for i in range(len(rv_train)):
    if id_review[i] in id_business:
        id_gym.append(i)
review_train = rv_train.loc[id_gym]

In [98]:
merge_train=business_train.merge(review_train)
merge_train.to_csv("merge_train.csv")

In [121]:
#Read review_test.json file.
rv_test = []
with open('review_test.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    rv_tes = json.loads(line)
                    rv_test.append(rv_tes)
                else:
                    break
        except:
            f.close()
rv_test = pd.DataFrame(rv_test)

In [144]:
# Read business_test.json file.
bs_test=[]
with open('business_test.json',"r") as f:
        try:
            while True:
                line = f.readline()
                if line:
                    bs_tes = json.loads(line)
                    bs_test.append(bs_tes)
                else:
                    break
        except:
            f.close()
bs_test = pd.DataFrame(bs_test)

In [149]:
#Pick up the gym data from business_test file.
categories=bs_test[['categories']]
pat_rest=re.compile(r'.*Gym.*')
num=[]
for row in tqdm(range(len(categories))):
    if categories.iloc[row]['categories'] is None:
        num.append(row)
    else:
        if not pat_rest.match(categories.iloc[row]['categories']):
            num.append(row)
bs_test = bs_test.drop(index = num)

100%|██████████| 38000/38000 [00:06<00:00, 6283.46it/s]


In [155]:
#Pick up the gym data from train_test file.
bs_id = list(bs_test['business_id'])
rv_id = rv_test['business_id']
id = []
for i in range(len(rv_id)):
    if rv_id[i] in bs_id:
        id.append(i)
rv_test = rv_test.loc[id]

In [161]:
merge_test = bs_test.merge(rv_test)

In [173]:
attr_split = list(dict(merge_test['attributes']).values())
attr = pd.DataFrame(attr_split[0],index=[0])
for i in range(1,len(attr_split)):
    attr = pd.concat([attr,pd.DataFrame(attr_split[i],index=[i])])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [177]:
merge_test=pd.concat([merge_test,attr],axis=1)
merge_test.to_csv("merge_test.csv")